<a target="_blank" href="https://colab.research.google.com/github/eeg2025/startkit/blob/main/challenge_2.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# FUCK IT , MY WAY OR THE HIGH WAY

## libs


In [1]:
import os
import shutil
from pathlib import Path
import pandas as pd
from tqdm import tqdm
import mne
import numpy as np


## preparing the full metadata data 

In [2]:
ALL_TASKS = ['contrastChangeDetection', 'DespicableMe', 'DiaryOfAWimpyKid', 'FunwithFractals', 'RestingState', 'seqLearning8target', 'surroundSupp', 'symbolSearch', 'ThePresent', 'seqLearning6target']

In [2]:
INITIAL_DATA_PATH = Path(r"D:\projects\pytorch_training\data_1")
FINAL_DATA_DIR = Path(r"C:\disque d\ai_stuff\projects\pytorchtraining\eeg_competition")
os.makedirs(FINAL_DATA_DIR, exist_ok=True)
full_meta_data_path = FINAL_DATA_DIR / Path("full_meta_data.csv")
full_meta_data =None
for release_path in os.listdir(INITIAL_DATA_PATH):

    release_path = INITIAL_DATA_PATH / release_path
    curr_meta_data_path = release_path / "participants.tsv"
    curr_meta_data = pd.read_csv(curr_meta_data_path, sep="\t")
    if full_meta_data is    None:
        full_meta_data = curr_meta_data
    else:
        full_meta_data = pd.concat([full_meta_data, curr_meta_data])

full_meta_data.to_csv(full_meta_data_path)
print(f"saved meta data at {full_meta_data_path}")


saved meta data at C:\disque d\ai_stuff\projects\pytorchtraining\eeg_competition\full_meta_data.csv


## AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA

In [4]:

RAW_DATA_PATH = FINAL_DATA_DIR / Path("raw_data_info")
PROCCESSED_SHARDS = FINAL_DATA_DIR / Path("processed_shards")
RELEASE_RELATED_DATA = FINAL_DATA_DIR / Path("release_related_data")

os.makedirs(RAW_DATA_PATH, exist_ok=True)
os.makedirs(PROCCESSED_SHARDS, exist_ok=True)
os.makedirs(RELEASE_RELATED_DATA, exist_ok=True)

FREQ = 100

WINDOW_LEN = 2 * FREQ
REQUIRED_NUM_CHANNELS = 129
MIN_REC_LEN = 60 * FREQ
shard_index = 0
for release in os.listdir(INITIAL_DATA_PATH):
    # for every release , we create a folder in the release related data
    release_path = INITIAL_DATA_PATH / release
    final_dir_release_path = RELEASE_RELATED_DATA / release
    os.makedirs(final_dir_release_path, exist_ok=True)
    for item in tqdm(os.listdir(release_path)):
        # if not a subject related data , we copy it to the release related data
        if not item.startswith("sub-"):
            src = release_path / item
            dst = final_dir_release_path / item

            if src.is_dir():
                shutil.copytree(src, dst, dirs_exist_ok=True)
            elif src.is_file():
                shutil.copy2(src, dst)

        else :
            # its a subject , lets copy its raw non bdf files to the raw data path
            subject_id = item
            subject_data_path = release_path / item / "eeg"
            for file in os.listdir(subject_data_path):
                try :
                    file_path = subject_data_path / file
                    # get details
                    name, ext = os.path.splitext(file)
                    parts = name.split("_")

                    # defaults
                    subject = None
                    task = None
                    run = "1"
                    ftype = None

                    for part in parts:
                        if part.startswith("sub-"):
                            subject = part
                        elif part.startswith("task-"):
                            task = part.replace("task-", "")
                        elif part.startswith("run-"):
                            run = part.replace("run-", "")
                        else:
                            ftype = part


                    if ext != ".bdf":
                        # not a eeg data , copy it to the right dir
                        task_final_data_path = RAW_DATA_PATH / task
                        os.makedirs(task_final_data_path, exist_ok=True)
                        subject_task_final_data_path = task_final_data_path / subject
                        os.makedirs(subject_task_final_data_path, exist_ok=True)
                        run_subject_task_final_data_path = subject_task_final_data_path / ("run-"+run)
                        os.makedirs(run_subject_task_final_data_path, exist_ok=True)
                        final_destination_path = run_subject_task_final_data_path / file
                        shutil.copy2(file_path, final_destination_path)
                    else:
                        # its a eeg data , lets proccess it 
                        eeg_file_proccessing_path = PROCCESSED_SHARDS / f"{release}_{task}_{subject}_run-{run}_shard-{shard_index}.npy"
                        raw = mne.io.read_raw_bdf(file_path, preload=True, verbose=False)
                        data = raw.get_data().astype(np.float32)
                        if data.shape[0] != REQUIRED_NUM_CHANNELS:
                            print(f"Skipping {file_path}, wrong channel count {data.shape[0]}")
                            continue
                        if data.shape[1] < MIN_REC_LEN:
                            print(f"Skipping {file_path}, not enough samples ({data.shape[1]})")
                            continue

                        n_windows = data.shape[1] // WINDOW_LEN 
                        cut_data = data[:, :n_windows * WINDOW_LEN] 
                        reshaped_data = cut_data.reshape(cut_data.shape[0], n_windows, WINDOW_LEN).transpose(1, 0, 2)

                        np.save(eeg_file_proccessing_path, reshaped_data.astype(np.float32))

                        del raw , data, cut_data, reshaped_data
                        shard_index += 1
                except Exception as e:
                    print(f"Error processing {file_path} : {e}")
                    continue

                    

                    


    print(f"✅ Finished release {release}")

                



  0%|          | 0/560 [00:00<?, ?it/s]

 12%|█▏        | 65/560 [04:30<35:09,  4.26s/it]  

Skipping D:\projects\pytorch_training\data_2\R10\sub-NDARCA851LUG\eeg\sub-NDARCA851LUG_task-FunwithFractals_eeg.bdf, not enough samples (5500)


 70%|██████▉   | 390/560 [26:25<07:02,  2.48s/it] 

Skipping D:\projects\pytorch_training\data_2\R10\sub-NDARTY128YLU\eeg\sub-NDARTY128YLU_task-RestingState_eeg.bdf, not enough samples (100)


...

Skipping D:\projects\pytorch_training\data_2\R9\sub-NDARJP304NK1\eeg\sub-NDARJP304NK1_task-RestingState_eeg.bdf, not enough samples (600)


 80%|████████  | 259/322 [37:46<07:46,  7.40s/it]

Skipping D:\projects\pytorch_training\data_2\R9\sub-NDARXJ896XFC\eeg\sub-NDARXJ896XFC_task-DespicableMe_eeg.bdf, not enough samples (200)


100%|██████████| 322/322 [44:16<00:00,  8.25s/it]

✅ Finished release R9


In [ ]:
import pandas as pd
from pathlib import Path
from sklearn.model_selection import train_test_split

def make_subject_splits(final_data_dir: Path, seed: int = 42,
                        test_size: float = 0.10, val_size: float = 0.10):
    """
    Split subjects listed in full_meta_data.csv (column 'participant_id')
    into train/val/test groups.
    """
    meta_path = final_data_dir / "full_meta_data.csv"
    if not meta_path.exists():
        raise FileNotFoundError(f"Missing {meta_path}")

    df = pd.read_csv(meta_path)
    if "participant_id" not in df.columns:
        raise ValueError("full_meta_data.csv must contain 'participant_id' column")

    subjects = sorted(df["participant_id"].unique())

    # first split off temp (val + test)
    train_subj, temp_subj = train_test_split(
        subjects,
        test_size=(test_size + val_size),
        random_state=seed,
        shuffle=True
    )

    # now split temp into val and test
    val_ratio = val_size / (test_size + val_size)
    val_subj, test_subj = train_test_split(
        temp_subj,
        test_size=(1 - val_ratio),
        random_state=seed,
        shuffle=True
    )

    splits = {
        "train": train_subj,
        "val": val_subj,
        "test": test_subj
    }



    return splits


In [ ]:
import os
import bisect
import numpy as np
from pathlib import Path
from torch.utils.data import Dataset, DataLoader

class EEGShardDataset_FULL(Dataset):
    def __init__(self, final_data_dir: Path, subject_list: list[str]):
        """
        EEG dataset loader for preprocessed shards.
        Builds an index for fast access by window index.
        """
        super().__init__()
        self.data_dir = Path(final_data_dir)
        self.shards_dir = self.data_dir / "processed_shards"
        self.subject_list = set(subject_list)

        self.files = []
        self.starts = []
        self.n_windows = []
        total_windows = 0

        # list all shard files
        all_files = sorted(
            f for f in os.listdir(self.shards_dir) if f.endswith(".npy")
        )

        for fname in tqdm(all_files):
            # only include subjects in our split
            # file pattern: release_task_subject_run-{run}_shard-{id}.npy
            if not any(subj in fname for subj in self.subject_list):
                continue

            fpath = self.shards_dir / fname
            try:
                arr = np.load(fpath, mmap_mode="r")
                n_w = arr.shape[0]
                self.files.append(fpath)
                self.starts.append(total_windows)
                self.n_windows.append(n_w)
                total_windows += n_w
                del arr
            except Exception as e:
                print(f"⚠️ Skipping {fname}: {e}")

        self.total_windows = total_windows
        print(f"✅ Loaded {len(self.files)} shards ({self.total_windows} windows total)")

    def __len__(self):
        return self.total_windows

    def __getitem__(self, index):
        """
        Locate which shard and local window correspond to the given global index.
        Returns (window_data, info_dict).
        """
        # find shard index
        shard_idx = bisect.bisect_right(self.starts, index) - 1
        local_idx = index - self.starts[shard_idx]

        shard_path = self.files[shard_idx]
        data = np.load(shard_path, mmap_mode="r")[local_idx]  # (channels, time)

        # --- parse filename directly ---
        # release_task_subject_run-{run}_shard-{id}.npy
        fname = shard_path.stem
        parts = fname.split("_")

        info = {
            "release": parts[0],
            "task": parts[1],
            "subject": parts[2],
            "run": parts[3].replace("run-", "")
        }

        return data, info


In [25]:
from pathlib import Path



splits = make_subject_splits(FINAL_DATA_DIR, seed=1337)

train_ds = EEGShardDataset_FULL(FINAL_DATA_DIR, splits["train"])
val_ds   = EEGShardDataset_FULL(FINAL_DATA_DIR, splits["val"])

print(len(train_ds))  
x, info = train_ds[1234]
print(x.shape)        
print(info)           


✅ Loaded 3708 shards (533861 windows total)
✅ Loaded 492 shards (67885 windows total)
533861
(129, 200)
{'release': 'R1', 'task': 'DespicableMe', 'subject': 'sub-NDARCR499NE4', 'run': '1'}
